In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('songdata.csv')
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [3]:
df.shape

(57650, 4)

In [4]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [5]:
df.shape

(5000, 3)

In [6]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [7]:
df['text'][0]

"there you go and baby here am i   well you left me here so i could sit and cry   golly gee what have you done to me   well i guess it doesn't matter anymore      do you remember baby last september   how you held me tight each and every night   oh baby how you drove me crazy   but i guess it doesn't matter anymore      there's no use in me a-crying   i've done everything now i'm sick of trying   i've thrown away my nights   wasted all my days over you      now you go your way baby and i'll go mine   now and forever till the end of time   i'll find somebody new and baby   we'll say we're through   and you won't matter anymore      there's no use in me a-crying   i've done everything now i'm sick of trying   i've thrown away my nights   wasted all my days over you      now you go your way baby and i'll go mine   now and forever till the end of time   and i'll find somebody new and baby   we'll say we're through   and you won't matter anymore   no you won't matter anymore   you won't mat

In [8]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [9]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [12]:
similarity[0]

array([1.        , 0.07829276, 0.17354231, ..., 0.01173347, 0.06414057,
       0.08753482])

In [16]:
df[df['song']==''].index[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

# recommedation function

In [17]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [18]:
recommendation('Alma Mater')

['Dream The Night Away',
 'Confidence',
 'Everything Happens For The Best',
 'Brand New Day',
 'Dream',
 'This Must Be The Place I Waited Years To Leave',
 'Brand New Car',
 'Anything Is Possible',
 'Blue Prelude',
 'Here Comes Yet Another Day',
 'Everything You Know Is Wrong',
 'Take Me To Your Heart',
 'New Shoes',
 'Not Giving Up',
 'The Dream Is Still Alive',
 "There's No Good In Goodbye",
 'Cygnet Committee',
 'Emenius Sleepus',
 "It's You",
 'Details In The Fabric']

In [19]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))